In [ ]:
!pip install texthero

     |████████████████████████████████| 235 kB 5.0 MB/s 
     |████████████████████████████████| 1.5 MB 43.1 MB/s 
     |████████████████████████████████| 748 kB 19.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import texthero as hero
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_json('drive/My Drive/news_dataset.json',lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [ ]:
#headline and short description data is related so combine the data
df['Text']=df['headline']+" "+df['short_description']
df

,category,headline,authors,link,short_description,date,Text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...
...,...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28,RIM CEO Thorsten Heins' 'Significant' Plans Fo...
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28,Maria Sharapova Stunned By Victoria Azarenka I...
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28,"Giants Over Patriots, Jets Over Colts Among M..."
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28,Aldon Smith Arrested: 49ers Linebacker Busted ...


In [ ]:
df=df[['Text','category']]
df

,Text,category
0,There Were 2 Mass Shootings In Texas Last Week...,CRIME
1,Will Smith Joins Diplo And Nicky Jam For The 2...,ENTERTAINMENT
2,Hugh Grant Marries For The First Time At Age 5...,ENTERTAINMENT
3,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,ENTERTAINMENT
4,Julianna Margulies Uses Donald Trump Poop Bags...,ENTERTAINMENT
...,...,...
200848,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH
200849,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS
200850,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS
200851,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS


In [ ]:
df2=pd.read_csv('drive/My Drive/BBCNewsTrain.csv')
df2.rename(columns = {'Category':'category'}, inplace = True)
df2=df2.iloc[:,1:]
df2

,Text,category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business
...,...,...
1485,double eviction from big brother model caprice...,entertainment
1486,dj double act revamp chart show dj duo jk and ...,entertainment
1487,weak dollar hits reuters revenues at media gro...,business
1488,apple ipod family expands market apple has exp...,tech


In [ ]:
df2.category=df2.category.map(lambda x:"BUSINESS" if x=="business" else x )
df2.category=df2.category.map(lambda x:"TECH" if x=="tech" else x )
df2.category=df2.category.map(lambda x:"POLITICS" if x=="politics" else x )
df2.category=df2.category.map(lambda x:"ENTERTAINMENT" if x=="entertainment" else x )
df2.category=df2.category.map(lambda x:"SPORTS" if x=="sport" else x )

In [ ]:
print(df2['category'].value_counts())

SPORTS           346
BUSINESS         336
POLITICS         274
ENTERTAINMENT    273
TECH             261
Name: category, dtype: int64


In [ ]:
entertainment=df2[df2['category']=='ENTERTAINMENT']#273
ENTERTAINMENT=df[df['category']=='ENTERTAINMENT'][:727]

politics=df2[df2['category']=='POLITICS']#274
POLITICS=df[df['category']=='POLITICS'][:726]

sport=df2[df2['category']=='SPORTS']#346
SPORT=df[df['category']=='SPORTS'][:654]

tech=df2[df2['category']=='TECH']#261
TECH=df[df['category']=='TECH'][:739]

business=df2[df2['category']=='BUSINESS']#336
BUSINESS=df[df['category']=='BUSINESS'][:664]

In [ ]:
rmvcls=['SPORTS','ARTS','ENVIRONMENT','SCIENCE','TRAVEL','CRIME','ARTS & CULTURE','BUSINESS','POLITICS','ENTERTAINMENT','TECH','LATINO VOICES','CULTURE & ARTS','BLACK VOICES','IMPACT','GREEN', 'TASTE', 'GOOD NEWS', 'FIFTY','STYLE','WOMEN', 'QUEER VOICES','MONEY','DIVORCE', 'WEDDINGS', 'PARENTING', 'HOME & LIVING', 'WORLD NEWS', 'WEIRD NEWS','COMEDY','MEDIA','RELIGION', 'COLLEGE', 'PARENTS','HEALTHY LIVING', 'THE WORLDPOST', 'WORLDPOST',]

In [ ]:
for i in range(len(rmvcls)):
  df=df[df['category']!=rmvcls[i]]
print(len(df.category.unique()))
df.category.unique()

4


array(['EDUCATION', 'WELLNESS', 'STYLE & BEAUTY', 'FOOD & DRINK'],
      dtype=object)

In [ ]:
betp=pd.concat([tech,TECH,entertainment,ENTERTAINMENT,politics,POLITICS,sport,SPORT,business,BUSINESS])

In [ ]:
df=pd.concat([betp,df])
df.reset_index(inplace = True,drop=True)


In [ ]:
df['Text'].isnull().sum()

0

In [ ]:
print(len(df.category.unique()))
print(df.category.unique())

print(df['category'].value_counts())

9
['TECH' 'ENTERTAINMENT' 'POLITICS' 'SPORTS' 'BUSINESS' 'EDUCATION'
 'WELLNESS' 'STYLE & BEAUTY' 'FOOD & DRINK']
WELLNESS          17827
STYLE & BEAUTY     9649
FOOD & DRINK       6226
EDUCATION          1004
TECH               1000
ENTERTAINMENT      1000
POLITICS           1000
SPORTS             1000
BUSINESS           1000
Name: category, dtype: int64


# Under Sampling

In [ ]:
y=df['category'].unique()
print(len(y))
print(y)

9
['TECH' 'ENTERTAINMENT' 'POLITICS' 'SPORTS' 'BUSINESS' 'EDUCATION'
 'WELLNESS' 'STYLE & BEAUTY' 'FOOD & DRINK']


In [ ]:
l=[]
#n=no.of records for each class
n=1000
for i in y:
    D=df[df['category']==i].iloc[:n,:]
    l.append(D)
    dff=pd.concat(l)
dff

,Text,category
0,lifestyle governs mobile choice faster bett...,TECH
1,moving mobile improves golf swing a mobile pho...,TECH
2,bt boosts its broadband packages british telec...,TECH
3,peer-to-peer nets here to stay peer-to-peer ...,TECH
4,pompeii gets digital make-over the old-fashion...,TECH
...,...,...
10716,How to Saber a Bottle of Champagne The key to ...,FOOD & DRINK
10729,Blood Orange Chocolate Mousse: Choc and Awe Th...,FOOD & DRINK
10741,9 Mind-Boggling Food Facts We Learned From Red...,FOOD & DRINK
10742,Rock Star Champagne Grower-Producers: Serge Ma...,FOOD & DRINK


In [ ]:
print(dff['category'].value_counts())

TECH              1000
ENTERTAINMENT     1000
POLITICS          1000
SPORTS            1000
BUSINESS          1000
EDUCATION         1000
WELLNESS          1000
STYLE & BEAUTY    1000
FOOD & DRINK      1000
Name: category, dtype: int64


In [ ]:
dff[dff['category']=='TECH']

,Text,category
0,lifestyle governs mobile choice faster bett...,TECH
1,moving mobile improves golf swing a mobile pho...,TECH
2,bt boosts its broadband packages british telec...,TECH
3,peer-to-peer nets here to stay peer-to-peer ...,TECH
4,pompeii gets digital make-over the old-fashion...,TECH
...,...,...
995,"How To Use Peach, Your New Social Media Obsess...",TECH
996,Unlimited Cellphone Plans Might Be Coming Back...,TECH
997,"Facebook Is Killing Phone Numbers, And That's ...",TECH
998,Uber Just Took A Huge Step Toward Integrating ...,TECH


In [ ]:
!pip install tensorflow-text 

     |████████████████████████████████| 4.9 MB 4.2 MB/s 
     |████████████████████████████████| 462 kB 10.8 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# USING BERT

In [ ]:
bert_preprocess = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")  
bert_encoder = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [ ]:

dff['Text'] = hero.clean(dff['Text'])


In [ ]:
vectors_1=[]
for i in y:
  T=dff[dff['category']==i]['Text'][:500]
  bp=bert_preprocess(T)
  be=bert_encoder(bp)['pooled_output']
  vectors_1.append(be)
print(len(vectors_1))
print(len(vectors_1[0]))
print(vectors_1[0])


9
500
tf.Tensor(
[[-0.71811736 -0.29613528  0.0204109  ... -0.03950692 -0.46839988
   0.6838525 ]
 [-0.6639139  -0.24776408  0.31306913 ...  0.10773205 -0.34708464
   0.56992114]
 [-0.719594   -0.44590586 -0.45840815 ... -0.12719172 -0.55573994
   0.6431317 ]
 ...
 [-0.8398725  -0.72090477 -0.96539545 ... -0.9000882  -0.70714355
   0.83730745]
 [-0.8646125  -0.56751835 -0.8798459  ... -0.7127086  -0.71338063
   0.88484156]
 [-0.89939487 -0.5427844  -0.7387086  ... -0.40879944 -0.6660037
   0.86907756]], shape=(500, 768), dtype=float32)


In [ ]:
vectors_2=[]
for i in y:
  T=dff[dff['category']==i]['Text'][500:]
  bp=bert_preprocess(T)
  be=bert_encoder(bp)['pooled_output']
  vectors_2.append(be)
print(len(vectors_2))
print(len(vectors_2[0]))
print(vectors_2[0])

9
500
tf.Tensor(
[[-0.7375261  -0.32195225 -0.42294118 ... -0.7375519  -0.20355599
   0.6688485 ]
 [-0.73201287 -0.25359616 -0.25521323 ... -0.20241445 -0.33447063
   0.5960446 ]
 [-0.7718002  -0.39519298 -0.21395539 ...  0.23507817 -0.6038068
   0.8700905 ]
 ...
 [-0.8605115  -0.55129063 -0.8621437  ... -0.5052788  -0.67196566
   0.8555567 ]
 [-0.67518485 -0.12461834 -0.43771052 ... -0.1524258  -0.512523
   0.73451805]
 [-0.67495644 -0.20808241  0.6057056  ...  0.48406813 -0.5003169
   0.6555964 ]], shape=(500, 768), dtype=float32)


In [ ]:
#1000 records... per category 500 and 500 formate[[500,500],[500,500]......]
Vectors_=[]
for i,j in enumerate(zip(vectors_1,vectors_2)):
    Vectors_.append(list(j))
print(len(Vectors_))
print(len(Vectors_[0]))


9
2


In [ ]:
Vectors_

In [ ]:
#formate [500,500,500,500....]
Vectors_18=[]
for i in range(len(Vectors_)):
  for j in range(len(Vectors_[i])):
    Vectors_18.append(Vectors_[i][j])
print(len(Vectors_18))
Vectors_18

In [ ]:
len(Vectors_18[0])

500

In [ ]:
z=[]
for i in range(len(Vectors_18)):
  for j in range(len(Vectors_18[0])):
    z.append(Vectors_18[i][j])

In [ ]:
len(z)

9000

In [ ]:
X=z
y=dff['category']

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(y)
le_name_mapping = dict(zip(le.transform(le.classes_),le.classes_))
print(le_name_mapping)

{0: 'BUSINESS', 1: 'EDUCATION', 2: 'ENTERTAINMENT', 3: 'FOOD & DRINK', 4: 'POLITICS', 5: 'SPORTS', 6: 'STYLE & BEAUTY', 7: 'TECH', 8: 'WELLNESS'}


In [ ]:
y=dff['category']

dff['category_id']=le.fit_transform(y)
y=dff['category_id']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,stratify=y,random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
ss=MinMaxScaler()
X_train_ss=ss.fit_transform(X_train)
X_test_ss=ss.transform(X_test)

In [ ]:
# saving standard scaler object
from joblib import dump,load
dump(ss, 'drive/My Drive/std_scaler.bin', compress=True)

['drive/My Drive/std_scaler.bin']

# Model checking with CrossValidation
- cross validation checks the prediction ability of a model

In [ ]:
models=[]


from sklearn.linear_model import LogisticRegression
logre = LogisticRegression(tol=1,max_iter=1000)
models.append(logre)

from sklearn.svm._classes import LinearSVC
lsvc=LinearSVC(tol=1,max_iter=1000)#tol is used to assess levels of multicollinearity. Generally, a tolerance is a threshold which, if crossed, stops the iterations of a solver.
models.append(lsvc)

from sklearn.linear_model import LogisticRegressionCV
logrecv=LogisticRegressionCV(tol=1,max_iter=1000)
models.append(logrecv)

from sklearn.svm import SVC
svc=SVC()
models.append(svc)

from sklearn.linear_model import SGDClassifier
sgd=SGDClassifier()
models.append(sgd)

from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
models.append(nb)

from sklearn.tree import  DecisionTreeClassifier
dt=DecisionTreeClassifier()
models.append(dt)

from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
models.append(rf)

from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
models.append(knn)

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
cv=KFold(10)
for model in models:
  score=cross_val_score(model,X_train_ss,y_train,cv=cv)
  print(str(model).split('(')[0]+' :- ',np.mean(score))


LogisticRegression :-  0.8188571428571428
LinearSVC :-  0.8125714285714285
LogisticRegressionCV :-  0.8171428571428571
SVC :-  0.766095238095238
SGDClassifier :-  0.7645714285714286
GaussianNB :-  0.3363809523809524
DecisionTreeClassifier :-  0.4135238095238095
RandomForestClassifier :-  0.6544761904761904
KNeighborsClassifier :-  0.5872380952380951


In [ ]:
from sklearn.linear_model import LogisticRegression
logre = LogisticRegression(tol=1,max_iter=1000)
logre.fit(X_train_ss,y_train)
print(accuracy_score(y_train,logre.predict(X_train_ss)))
print(accuracy_score(y_test,logre.predict(X_test_ss)))

0.8549629629629629
0.8093333333333333


In [ ]:
from sklearn.linear_model import LogisticRegressionCV
logrecv=LogisticRegressionCV(tol=1,max_iter=1000)
logrecv.fit(X_train_ss,y_train)
print(accuracy_score(y_train,logrecv.predict(X_train_ss)))
print(accuracy_score(y_test,logrecv.predict(X_test_ss)))

0.8717037037037038
0.8097777777777778


In [ ]:
from sklearn.svm._classes import LinearSVC
lsvc=LinearSVC(tol=1,max_iter=1000)
lsvc.fit(X_train_ss,y_train)
print(accuracy_score(y_train,lsvc.predict(X_train_ss)))
print(accuracy_score(y_test,lsvc.predict(X_test_ss)))

0.8974814814814814
0.8173333333333334


In [ ]:
ss=load('drive/My Drive/std_scaler.bin')


In [ ]:
#NOTE:- do not creat sclaer objects here just use transofrm fun 
text=input('enter new headline:-')
d_ = pd.DataFrame([text], columns = ['txt'])
text=hero.clean(d_['txt'])#d_['txt'].pipe(hero.clean, custom_pipeline)
bp_=bert_preprocess(text)
vectors_=bert_encoder(bp_)['pooled_output']
vec=ss.transform(vectors_)
prediction=logre.predict(vec)
print(le_name_mapping[prediction[0]])


enter new headline:-meghana bought a new smartphone
TECH
